In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,80.60,78,75,5.75,light rain,2021-02-01 03:39:33
1,1,Bengkulu,ID,-3.8004,102.2655,85.46,59,77,5.99,broken clouds,2021-02-01 03:39:39
2,2,Adrar,MR,20.5022,-10.0711,62.76,28,99,7.54,overcast clouds,2021-02-01 03:40:42
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,-27.40,75,20,3.44,few clouds,2021-02-01 03:39:36
4,4,Monki,PL,53.4050,22.7979,12.99,88,20,7.96,few clouds,2021-02-01 03:40:42


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.


In [ ]:
# 4a. Determine if there are any empty rows.


In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)